# Picking apart a floating point number

Copyright (C) 2020 Andreas Kloeckner

<details>
<summary>MIT License</summary>
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
</details>

In [1]:
import numpy as np

In [31]:
# Never mind the details of this function...

def left_pad(s, desired_len, pad="0"):
    if len(s) < desired_len:
        s = (desired_len - len(s)) * pad + s
    return s
        
def pretty_print_fp(x):
    print("---------------------------------------------")
    print("Floating point structure for %r" % x)
    print("---------------------------------------------")
    import struct
    s = struct.pack("d", x)

    def get_bit(i):
        byte_nr, bit_nr = divmod(i, 8)
        return int(bool(
            s[byte_nr] & (1 << bit_nr)
            ))

    def get_bits(lsb, count):
        return sum(get_bit(i+lsb)*2**i for i in range(count))

    # https://en.wikipedia.org/wiki/Double_precision_floating-point_format

    print("Sign bit (1:negative):", get_bit(63))
    exponent = get_bits(52, 11)
    print("Stored exponent bits: %d" % exponent)
    print("Exponent (with offset): %d" % (exponent - 1023))
    if exponent == 0:
        print("  In subnormal range. Effective exponent: -1022")
    fraction = get_bits(0, 52)
    print(f"Stored fraction bits: ({left_pad(bin(fraction)[2:], 52)}_2")
    if exponent != 0:
        significand = fraction + 2**52
    else:
        significand = fraction
    sig_str = left_pad(bin(significand)[2:], 53)
    print(f"Significand: ({sig_str[0]}.{sig_str[1:]})_2")
    print("Significand:", repr(significand / (2**52)))

In [32]:
pretty_print_fp(2**(-1025))

---------------------------------------------
Floating point structure for 2.781342323134e-309
---------------------------------------------
Sign bit (1:negative): 0
Stored exponent bits: 0
Exponent (with offset): -1023
  In subnormal range. Effective exponent: -1022
Stored fraction bits: (0010000000000000000000000000000000000000000000000000_2
Significand: (0.0010000000000000000000000000000000000000000000000000)_2
Significand: 0.125


Things to try:

* Twiddle the sign bit
* 1,2,4,8
* 0.5,0.25
* $2^{\pm 1023}$, $2^{\pm 1024}$
* `float("nan")`